In [39]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Load the Point Cloud ---
# The filename has been updated to 'outer.ply'
try:
    pcd = o3d.io.read_point_cloud("outer.ply")
    if not pcd.has_points():
        raise ValueError("The loaded PLY file contains no points.")
    print(f"Successfully loaded 'outer.ply' with {len(pcd.points)} points.")
except Exception as e:
    print(f"Error loading point cloud: {e}")
    print("Please ensure 'outer.ply' exists in the same directory as your Jupyter notebook.")
    # Create a dummy point cloud for demonstration if loading fails
    print("Generating a dummy point cloud for demonstration purposes.")
    np.random.seed(42)
    points = np.random.rand(1000, 3) * 10
    # Make it a bit elongated and rotated to show OBB advantage
    points[:, 0] = points[:, 0] * 2 + points[:, 1] * 0.5
    points[:, 1] = points[:, 1] * 1.5
    # Apply a rotation
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz((np.pi/4, np.pi/6, np.pi/8))
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.dot(points, rotation_matrix.T))


# --- 2. Generate the Oriented Bounding Box (OBB) ---
# This computes the tightest-fitting box around the points.
obb = pcd.get_oriented_bounding_box()


# --- 3. Visualize the Point Cloud and OBB ---

# Set OBB color to green for visibility
obb.color = (0, 1, 0) 

print("\nVisualizing the point cloud with its Oriented Bounding Box (Green).")
# This opens an interactive 3D window
o3d.visualization.draw_geometries([pcd, obb])



Successfully loaded 'outer.ply' with 10840 points.

Visualizing the point cloud with its Oriented Bounding Box (Green).


In [40]:
# --- 4. Locate and Highlight a Specific OBB Plane ---

print("\nIdentifying the correct side plane for visualization...")

# Get the geometric properties from the calculated OBB
obb_center = obb.center
obb_rotation = obb.R
obb_extent = obb.extent

# To find the correct face, we sort the OBB's dimensions (extents)
# sorted_indices[0] = index of the smallest dimension (thickness)
# sorted_indices[1] = index of the middle dimension (width)
# sorted_indices[2] = index of the largest dimension (length)
sorted_indices = np.argsort(obb_extent)

# --- KEY CHANGE IS HERE ---
# Instead of the middle axis (width), we now choose the smallest axis (thickness)
# to get the other perpendicular face.
plane_axis_idx = sorted_indices[0] # Changed from sorted_indices[1]

# The normal vector of the plane is the axis vector from the rotation matrix
normal_vector = obb_rotation[:, plane_axis_idx]

# The extent (thickness) of the box along this axis
plane_thickness = obb_extent[plane_axis_idx]

# Calculate the center point of the specific face we want to highlight.
# We start at the OBB's center and move half the thickness along the normal vector.
face_center = obb_center - normal_vector * (plane_thickness / 2.0)


# --- 5. Create a Visual Mesh for the Plane ---

# To visualize the plane, we create a very thin but large box mesh.
# We create a copy of the extents to modify for our visual mesh.
plane_mesh_extent = np.copy(obb_extent)

# First, we make the mesh flat by squashing the dimension along our normal vector.
plane_mesh_extent[plane_axis_idx] = 0.01

# Next, we enlarge the other two dimensions (width and length) to make the plane visible.
other_indices = [i for i in range(3) if i != plane_axis_idx]
plane_mesh_extent[other_indices] *= 1.2

# Create a new OBB that represents our visual plane
plane_obb = o3d.geometry.OrientedBoundingBox(face_center, obb_rotation, plane_mesh_extent)

# Create the actual mesh from this plane-like OBB
plane_mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(plane_obb)

# Give the plane a distinct color (e.g., blue)
plane_mesh.paint_uniform_color([0.1, 0.2, 0.8])

# --- 6. Final Visualization ---

print("Displaying Point Cloud, OBB (Green), and the corrected Highlighted Plane (Blue).")
o3d.visualization.draw_geometries([pcd, obb, plane_mesh])


Identifying the correct side plane for visualization...
Displaying Point Cloud, OBB (Green), and the corrected Highlighted Plane (Blue).


In [41]:
# --- 7. Mirror the Point Cloud Using the Selected Plane ---

print("\nMirroring the point cloud across the highlighted plane...")

# The two key components of our mirror plane are already defined:
# 'face_center' (a point on the plane, P₀)
# 'normal_vector' (the normal of the plane, n)

# We get the original points as a NumPy array
points = np.asarray(pcd.points)

# This is the vectorized formula for reflection: P' = P - 2 * dot(P - P₀, n) * n
# 1. Calculate vectors from each point to the plane
vecs_to_plane = points - face_center
# 2. Calculate the dot product (projection) of these vectors onto the normal
# This gives the distance of each point to the plane along the normal
dists = np.dot(vecs_to_plane, normal_vector)
# 3. Apply the formula to get the new mirrored points
# We scale the normal vector by twice the distance and subtract it from the original points
mirrored_points = points - 2 * np.outer(dists, normal_vector)

# --- 8. Create and Visualize the Mirrored Point Cloud ---

# Create a new Open3D point cloud object for the mirrored data
mirrored_pcd = o3d.geometry.PointCloud()
mirrored_pcd.points = o3d.utility.Vector3dVector(mirrored_points)

# Give the new mirrored cloud a distinct color (e.g., magenta)
mirrored_pcd.paint_uniform_color([1, 0, 1])

# Final visualization including the original cloud, OBB, plane, and the new mirrored cloud
print("Displaying Original PCD, OBB (Green), Plane (Blue), and Mirrored PCD (Magenta).")
o3d.visualization.draw_geometries([pcd, obb, plane_mesh, mirrored_pcd])


Mirroring the point cloud across the highlighted plane...
Displaying Original PCD, OBB (Green), Plane (Blue), and Mirrored PCD (Magenta).


In [42]:
# --- 9. Load the Reference Point Cloud ---

try:
    # Load the new reference point cloud
    ref_pcd = o3d.io.read_point_cloud("middle.ply")
    if not ref_pcd.has_points():
        raise ValueError("The loaded PLY file 'middle.ply' contains no points.")
    print(f"Successfully loaded 'middle.ply' with {len(ref_pcd.points)} points.")

    # Give the reference cloud a distinct color (e.g., yellow)
    ref_pcd.paint_uniform_color([1, 1, 0])

    # Visualize the original 'outer.ply' (pcd) and the new 'middle.ply' (ref_pcd) together
    print("Displaying original 'outer.ply' and reference 'middle.ply' (Yellow).")
    o3d.visualization.draw_geometries([pcd, ref_pcd])

except Exception as e:
    print(f"Error: {e}")
    print("Please ensure 'middle.ply' is in the same directory.")

Successfully loaded 'middle.ply' with 29014 points.
Displaying original 'outer.ply' and reference 'middle.ply' (Yellow).


In [43]:
# --- 10. Denoise and Downsample Both Point Clouds ---

print("Starting preprocessing...")
print(f"Before -> Mirrored cloud points: {len(mirrored_pcd.points)}")
print(f"Before -> Reference cloud points: {len(ref_pcd.points)}")

# --- Step 1: Denoising using Statistical Outlier Removal ---
# This removes sparse outlier points.
# nb_neighbors: How many neighbors to consider for the average distance calculation.
# std_ratio: The standard deviation multiplier. A smaller value makes the filter more aggressive.
print("\nStep 1: Denoising clouds...")
mirrored_denoised, _ = mirrored_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
ref_denoised, _ = ref_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)


# --- Step 2: Downsampling using Voxel Grid ---
# This reduces the number of points while preserving the shape by averaging points within each 3D box (voxel).
#
# !!! IMPORTANT !!!
# You MUST tune the 'voxel_size'. It depends on the scale of your object.
# - A LARGER voxel_size (e.g., 0.05) will downsample more aggressively (fewer points).
# - A SMALLER voxel_size (e.g., 0.005) will result in a denser cloud (more points).
# Start with a value like 0.01 and adjust it based on your results.
voxel_size = 0.01
print(f"Step 2: Downsampling clouds with a voxel size of {voxel_size}...")
mirrored_processed = mirrored_denoised.voxel_down_sample(voxel_size)
ref_processed = ref_denoised.voxel_down_sample(voxel_size)

print("\nPreprocessing complete.")
print(f"After -> Mirrored cloud points: {len(mirrored_processed.points)}")
print(f"After -> Reference cloud points: {len(ref_processed.points)}")


# --- Final Visualization of Processed Clouds ---
print("\nDisplaying the processed mirrored (Magenta) and reference (Yellow) clouds.")
o3d.visualization.draw_geometries([mirrored_processed, ref_processed])

Starting preprocessing...
Before -> Mirrored cloud points: 10840
Before -> Reference cloud points: 29014

Step 1: Denoising clouds...
Step 2: Downsampling clouds with a voxel size of 0.01...

Preprocessing complete.
After -> Mirrored cloud points: 2190
After -> Reference cloud points: 3613

Displaying the processed mirrored (Magenta) and reference (Yellow) clouds.


In [44]:
# --- 10.5: Unify Normals for Correct ICP Alignment ---

# IMPORTANT: Run this cell AFTER preprocessing and BEFORE registration.
# Define source and target from your preprocessed clouds.
source_to_fix = mirrored_processed
target_to_fix = ref_processed

print("Re-orienting normals for accurate registration...")

# 1. Estimate normals for both clouds
# (Even if done later in ICP, we do it here to orient them)
source_to_fix.estimate_normals()
target_to_fix.estimate_normals()

# 2. Orient the normals consistently
# This function makes all normals point towards a virtual camera at [0,0,0].
# This is a common method to make normal directions consistent.
source_to_fix.orient_normals_towards_camera_location([0, 0, 0])
target_to_fix.orient_normals_towards_camera_location([0, 0, 0])

print("Normals have been re-oriented.")

# Now, you can proceed to the registration cell (Cell 11).
# The variables 'mirrored_processed' and 'ref_processed' have been updated with correct normals.

Re-orienting normals for accurate registration...
Normals have been re-oriented.


# --- 11. Registration using Custom Fingerprinting ---
import time
from itertools import combinations

# --- Step 0: Define Source and Target for clarity ---
source = mirrored_processed
target = ref_processed

# --- Helper function for visualization (assuming it's in a previous cell) ---
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 1]) # Source is Magenta
    target_temp.paint_uniform_color([1, 1, 0]) # Target is Yellow
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

# --- Step 1: Your Fingerprinting Logic, Optimized with KNN ---
def get_robust_fingerprints_knn(pcd, num_neighbors):
    """
    Calculates a robust fingerprint for each point based on the perimeters of
    triangles formed with its K-Nearest Neighbors.
    """
    fingerprints = []
    points = np.asarray(pcd.points)
    kdtree = o3d.geometry.KDTreeFlann(pcd)
    
    for i in range(len(points)):
        ref_point = points[i]
        
        # Find K nearest neighbors for the current point
        [k, idx, _] = kdtree.search_knn_vector_3d(ref_point, num_neighbors + 1)
        neighbor_points = points[idx[1:]] # Exclude the point itself
        
        if len(neighbor_points) < 2:
            fingerprints.append([]) # Not enough neighbors to form a triangle
            continue
        
        perimeters = []
        # Create all unique pairs of neighbors to form triangles
        for n1, n2 in combinations(neighbor_points, 2):
            d1 = np.linalg.norm(ref_point - n1)
            d2 = np.linalg.norm(ref_point - n2)
            d3 = np.linalg.norm(n1 - n2)
            perimeters.append(d1 + d2 + d3)
        
        # The final signature is the sorted list of perimeters
        fingerprints.append(sorted(perimeters))
        
    return fingerprints

# --- Step 2: Find Corresponding Point Pairs ---
def find_correspondences(fingerprints1, fingerprints2):
    """Compares the robust fingerprints to find matching pairs."""
    correspondences = []
    unmatched2_indices = list(range(len(fingerprints2)))

    for i, sig1 in enumerate(fingerprints1):
        if not sig1: continue # Skip if no fingerprint was generated

        best_match_idx = -1
        min_diff = float('inf')

        # Find the best matching fingerprint in the target set
        for j in unmatched2_indices:
            sig2 = fingerprints2[j]
            if not sig2 or len(sig1) != len(sig2): continue
            
            diff = np.linalg.norm(np.array(sig1) - np.array(sig2))
            if diff < min_diff:
                min_diff = diff
                best_match_idx = j
        
        # Check if the match is good enough (using a tolerance)
        if best_match_idx != -1 and np.allclose(fingerprints1[i], fingerprints2[best_match_idx], atol=0.1):
            correspondences.append([i, best_match_idx])
            unmatched2_indices.remove(best_match_idx)

    return np.array(correspondences)


# --- Step 3: Run the Pairing and Estimate Transformation ---

# Display initial alignment
print("1. Displaying initial alignment before custom registration.")
draw_registration_result(source, target, np.identity(4))

# --- Parameter to Tune ---
# How many neighbors to use for creating the fingerprint.
# This is a key parameter to adjust based on your point cloud's density.
NUM_NEIGHBORS_FOR_FP = 10 

print(f"2. Calculating fingerprints using {NUM_NEIGHBORS_FOR_FP} nearest neighbors...")
start_time = time.time()
fp1 = get_robust_fingerprints_knn(source, NUM_NEIGHBORS_FOR_FP)
fp2 = get_robust_fingerprints_knn(target, NUM_NEIGHBORS_FOR_FP)
print(f"   Fingerprinting took {time.time() - start_time:.2f} seconds.")

print("\n3. Finding correspondences between point clouds...")
correspondences = find_correspondences(fp1, fp2)
print(f"   Found {len(correspondences)} corresponding pairs.")

if len(correspondences) < 3:
    print("\n❌ Not enough correspondences found to calculate a transformation. Try adjusting parameters.")
else:
    # --- Step 4: Calculate Transformation from Correspondences ---
    print("\n4. Estimating transformation from found pairs...")
    
    # Use Open3D's standard function to compute the best transformation from pairs
    corres_vector = o3d.utility.Vector2iVector(correspondences)
    estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    transformation_matrix = estimation.compute_transformation(source, target, corres_vector)

    # --- Step 5: Final Result ---
    print("\n5. Displaying final alignment after custom registration.")
    draw_registration_result(source, target, transformation_matrix)

    print("\nRegistration complete.")
    print("Final Transformation Matrix:")
    print(transformation_matrix)

# --- 11c. Hybrid Registration with Custom Python RANSAC ---
import time
import copy
from itertools import combinations
import numpy as np
import open3d as o3d


# --- Helper functions (no changes here) ---

def get_robust_fingerprints_knn(pcd, num_neighbors):
    """Calculates a fingerprint for each point using its K-Nearest Neighbors."""
    fingerprints = []
    points = np.asarray(pcd.points)
    kdtree = o3d.geometry.KDTreeFlann(pcd)
    for i in range(len(points)):
        ref_point = points[i]
        [k, idx, _] = kdtree.search_knn_vector_3d(ref_point, num_neighbors + 1)
        neighbor_points = points[idx[1:]]
        if len(neighbor_points) < 2:
            fingerprints.append(None)
            continue
        perimeters = [np.linalg.norm(ref_point - n1) + np.linalg.norm(ref_point - n2) + np.linalg.norm(n1 - n2) 
                      for n1, n2 in combinations(neighbor_points, 2)]
        fingerprints.append(sorted(perimeters))
    return fingerprints

def find_best_matches(fp1, fp2):
    """Finds the best potential match in fp2 for each fingerprint in fp1."""
    correspondences = []
    for i, sig1 in enumerate(fp1):
        if not sig1: continue
        best_match_j = -1
        min_diff = float('inf')
        for j, sig2 in enumerate(fp2):
            if not sig2 or len(sig1) != len(sig2): continue
            diff = np.linalg.norm(np.array(sig1) - np.array(sig2))
            if diff < min_diff:
                min_diff = diff
                best_match_j = j
        if best_match_j != -1:
            correspondences.append([i, best_match_j])
    return np.asarray(correspondences, dtype=np.int32)

def draw_registration_result(source, target, transformation):
    """Visualizes the registration result."""
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 1]) # Source is Magenta
    target_temp.paint_uniform_color([1, 1, 0]) # Target is Yellow
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

# --- NEW CUSTOM RANSAC FUNCTION ---
def run_python_ransac(source_pcd, target_pcd, corres_np, distance_threshold, iterations=1000):
    """
    Performs RANSAC in pure Python to avoid the broken Open3D function.
    """
    best_inlier_count = 0
    best_transformation = np.identity(4)
    
    source_points = np.asarray(source_pcd.points)
    target_points = np.asarray(target_pcd.points)
    
    estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    for i in range(iterations):
        # 1. Randomly sample 3 correspondence pairs
        sample_indices = np.random.choice(len(corres_np), 3, replace=False)
        sample_corres = corres_np[sample_indices]
        
        # 2. Get the 3D points for these pairs
        p_source = o3d.geometry.PointCloud()
        p_source.points = o3d.utility.Vector3dVector(source_points[sample_corres[:, 0]])
        p_target = o3d.geometry.PointCloud()
        p_target.points = o3d.utility.Vector3dVector(target_points[sample_corres[:, 1]])
        
        # 3. Estimate a transformation from this small sample
        current_transformation = estimation.compute_transformation(p_source, p_target, o3d.utility.Vector2iVector(np.array([[0,0],[1,1],[2,2]])))
        
        # 4. Count inliers: check how many of ALL pairs agree with this transformation
        source_transformed = copy.deepcopy(source_pcd)
        source_transformed.transform(current_transformation)
        distances = np.linalg.norm(np.asarray(source_transformed.points)[corres_np[:, 0]] - target_points[corres_np[:, 1]], axis=1)
        inlier_count = np.sum(distances < distance_threshold)
        
        # 5. If this is the best model so far, save it
        if inlier_count > best_inlier_count:
            best_inlier_count = inlier_count
            best_transformation = current_transformation
            
    # Create a RegistrationResult object to mimic the original function's output
    reg_result = o3d.pipelines.registration.RegistrationResult()
    reg_result.transformation = best_transformation
    return reg_result

# --- Main Registration Workflow ---

# Step 0: Define Source and Target
source = mirrored_processed 
target = ref_processed

# Step 1: Initial Visualization
print("1. Displaying initial alignment before registration.")
draw_registration_result(source, target, np.identity(4))

# Step 2: Generate potential correspondences
NUM_NEIGHBORS_FOR_FP = 10 
print(f"\n2. Calculating fingerprints using {NUM_NEIGHBORS_FOR_FP} nearest neighbors...")
start_time = time.time()
fp_source = get_robust_fingerprints_knn(source, NUM_NEIGHBORS_FOR_FP)
fp_target = get_robust_fingerprints_knn(target, NUM_NEIGHBORS_FOR_FP)
print(f"   Fingerprinting took {time.time() - start_time:.2f} seconds.")

print("\n3. Finding potential correspondences...")
potential_corres_np = find_best_matches(fp_source, fp_target)
print(f"   Found {len(potential_corres_np)} potential pairs.")

# Step 3: Global Registration using our new CUSTOM RANSAC function
distance_threshold_ransac = voxel_size * 5 
print(f"\n4. Running CUSTOM Python RANSAC to find the best transformation (threshold: {distance_threshold_ransac:.3f})...")
ransac_result = run_python_ransac(source, target, potential_corres_np, distance_threshold_ransac)

print("5. Displaying alignment after RANSAC.")
draw_registration_result(source, target, ransac_result.transformation)

# Step 4: Local Refinement with ICP
print("\n6. Running ICP to refine the alignment...")
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
distance_threshold_icp = voxel_size * 0.4
icp_result = o3d.pipelines.registration.registration_icp(
    source, target, distance_threshold_icp, ransac_result.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

# Step 5: Final Result
print("\n7. Displaying final alignment after ICP refinement.")
draw_registration_result(source, target, icp_result.transformation)

print("\nHybrid Registration complete.")
print("Final Transformation Matrix:")
print(icp_result.transformation)
print(f"Fitness: {icp_result.fitness:.4f}")
print(f"Inlier RMSE: {icp_result.inlier_rmse:.4f}")

In [45]:
# --- 11d. Standard Registration (FPFH + RANSAC + ICP) ---
import copy
import numpy as np
import open3d as o3d

# --- Helper Function ---
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 1]) # Source is Magenta
    target_temp.paint_uniform_color([1, 1, 0]) # Target is Yellow
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

# --- Main Registration Workflow ---

# Use the preprocessed clouds with UNIFIED NORMALS from the previous step
source = mirrored_processed 
target = ref_processed

# --- PARAMETERS TO TUNE ---
# This is the most important parameter. It defines the "neighborhood size" for feature calculation. 
# It should be larger than the 'voxel_size' used for preprocessing.
# Try increasing it (e.g., to 0.04, 0.05) or decreasing it to find the best value.
voxel_size_feature = 0.02

# The RANSAC distance threshold should be proportional to the feature voxel size.
distance_threshold_ransac = voxel_size_feature * 0.5


print(f"Starting standard registration with feature voxel size: {voxel_size_feature}")

# --- Global Registration (RANSAC on FPFH Features) ---

# Downsample for feature calculation
source_down = source.voxel_down_sample(voxel_size_feature)
target_down = target.voxel_down_sample(voxel_size_feature)

# Estimate normals
source_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size_feature * 2, max_nn=30))
target_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size_feature * 2, max_nn=30))

# Compute FPFH features
source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size_feature * 5, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size_feature * 5, max_nn=100))

# Run RANSAC
ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, True,
    distance_threshold_ransac,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 4, [
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold_ransac)
    ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 0.999))

print("Displaying alignment after RANSAC...")
draw_registration_result(source, target, ransac_result.transformation)

# --- Local Refinement (ICP) ---
print("Running ICP to refine the alignment...")
distance_threshold_icp = voxel_size * 0.4 
icp_result = o3d.pipelines.registration.registration_icp(
    source, target, distance_threshold_icp, ransac_result.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

print("Displaying final alignment after ICP...")
draw_registration_result(source, target, icp_result.transformation)

print("\nStandard Registration complete.")
print("Final Transformation Matrix:")
print(icp_result.transformation)
print(f"Fitness: {icp_result.fitness:.4f}")
print(f"Inlier RMSE: {icp_result.inlier_rmse:.4f}")

Starting standard registration with feature voxel size: 0.02
Displaying alignment after RANSAC...
Running ICP to refine the alignment...
Displaying final alignment after ICP...

Standard Registration complete.
Final Transformation Matrix:
[[ 0.99230326 -0.03428519  0.11899059  0.15936217]
 [ 0.04187104  0.99720692 -0.06184804 -0.07267867]
 [-0.11653777  0.06635427  0.99096723  0.00441674]
 [ 0.          0.          0.          1.        ]]
Fitness: 0.1667
Inlier RMSE: 0.0030
